### '###' 검색하여 문제점 수정할 것

### import

In [4]:
import my_utils as mu
import pandas as pd
import time
import json

from tqdm import tqdm

### 사용 함수들

In [5]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"').replace('True', '1').replace('False', '0'))


def parseRawdata(rawdata: pd.DataFrame, parse_to: int, use_tqdm: bool=False) -> pd.DataFrame:
    """
    `parse_to`: 파싱 대상 컬럼의 시작 인덱스\n
    `use_tqdm`: tqdm 출력의 사용 여부
    """
    matches = {a:[] for a in rawdata.columns[parse_to:]}

    if use_tqdm: parse_data = tqdm(rawdata.iloc)
    else: parse_data = rawdata.iloc

    for rec in parse_data:
        for col in [a for a in rawdata.columns][parse_to:]:
            matches[col].append(rawdataLoader(rec[col]))
    for col in [a for a in rawdata.columns][parse_to:]:
        rawdata[col] = matches[col]
    return rawdata


def noneDropper(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    라인 정보에 문제가 있는 레코드를 제거하는 함수\n
    `dataframe`에 `lane` 컬럼 필요
    """
    return dataframe.drop(dataframe[dataframe['lane']=='NONE'].index).reset_index(drop=True)


def ripDropper(dataframe: pd.DataFrame, rip_cut: int=50) -> pd.DataFrame:
    """
    표본수가 너무 적은 레코드를 제거하는 함수\n
    `dataframe`에 `game_count` 컬럼 필요
    """
    return dataframe.drop(dataframe[dataframe['game_count'] < rip_cut].index).reset_index(drop=True)


def winRateAdder(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    챔피언, 라인별 승률을 추가하는 함수\n
    `dataframe`에 `win`, `game_count` 컬럼 필요
    """
    dataframe['win_rate'] = round(dataframe['win'] / dataframe['game_count'] * 100, 2)
    return dataframe


def pickRateAdder(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    챔피언, 라인별 픽률을 추가하는 함수\n
    `dataframe`에 `champ_id`, `lane`, `game_count` 컬럼 필요
    """
    champ_id = dataframe['champ_id'].tolist()
    lane = dataframe['lane'].tolist()
    game_count = dataframe['game_count'].tolist()
    
    ### tolist()와 Series 사용 간의 속도 차이 검증 필요.
    dataframe['pick_rate'] = [round(game_count[idx] / (dataframe[(dataframe['champ_id']==cid)&(dataframe['lane']==lane[idx])]['game_count'].sum()) * 100, 2)
                              for idx, cid in enumerate(champ_id)]
    return dataframe

### 테스트용 DB 데이터 파싱

In [3]:
need_cols = ['VERSION','GAME_ID','PARTICIPANT_NUMBER','CHAMPION_ID','LANE','GAME','CHAMPION','SPELL','SKILLTREE','RUNE','STARTITEM','ITEMTREE','ITEM','KDA','DAMAGE']
version = '13.9.506.4846'
update_time = time.strftime('%Y-%m-%d %H:%M:%S')

# mu.rawdataIntegrityKeeper()
rawdata = mu.oracle_totalExecute(f"SELECT {','.join(need_cols)} FROM RAWDATA WHERE VERSION = '{version}'", debug_print=False)
rawdata.sort_values(['GAME_ID', 'PARTICIPANT_NUMBER'], ascending=[False, True], ignore_index=True, inplace=True)
rawdata = parseRawdata(rawdata, 5)
print('총 레코드 수:', len(rawdata))

총 레코드 수: 160590


## 인터벌 테이블 리스트

### 챔피언/초기 정보 (완료!)

In [6]:
def getChampionInfo(rawdata: pd.DataFrame, version: str, update_time: str) -> pd.DataFrame:

    # 속도 개선을 위한 리스트화
    gameid_list = rawdata['GAME_ID'].tolist()
    champid_list = rawdata['CHAMPION_ID'].tolist()
    lane_list = rawdata['LANE'].tolist()
    win_list = [int(a['win']) for a in rawdata['CHAMPION'].tolist()]
    ban_list = [[b['championId'] for b in a['bans']] for a in rawdata['GAME'].tolist()]

    # 필요 데이터 추출
    need_datas = [{
        'version': version,
        'update_time': update_time,
        'game_id': game_id,
        'champ_id': champid_list[idx],
        'lane': lane_list[idx],
        'win': win_list[idx],
        'bans': ban_list[idx],
        'game_count': 1
    } for idx, game_id in enumerate(gameid_list)]
    champ_info = pd.DataFrame(need_datas)

    # 그룹화
    result = champ_info.groupby(['version','update_time','champ_id','lane']).sum().reset_index()

    # 라인 정보에 문제가 있는 레코드 제거
    result = noneDropper(result)
    # 표본수가 부족한 레코드 제거
    # result = ripDropper(result, 50)
    # 승률 정보 추가
    result = winRateAdder(result)
    # 픽률 정보 추가
    result['pick_rate'] = round((result['game_count'] / len(rawdata))*1000, 2)

    # 밴률 정보 추가
    champs = mu.oracle_totalExecute('SELECT CHAMP_ID FROM CHAMPDATA', debug_print=False)
    champs = {a:0 for a in champs['CHAMP_ID']}
    champs[-1] = 0
    for idx in range(0, len(rawdata), 10):
        for cid in set(ban_list[idx]):
            champs[cid] += 1
    result['ban_rate'] = result['champ_id'].apply(lambda x: round(champs[x]/len(champ_info)*1000, 2))

    ### 티어 산출 알고리즘 적용 필요!
    result['tier'] = 0

    result.drop(result[result['game_count'] < 9].index, inplace=True)

    result_cols = [
    'version','update_time','champ_id','lane',
    'tier','win_rate','game_count','pick_rate','ban_rate']

    return result[result_cols]

### 챔피언/룬 (리워크 필요!!!)

In [5]:
def getChampionRune(rawdata: pd.DataFrame, version: str, update_time: str) -> pd.DataFrame:
    
    # 속도 개선을 위한 리스트화
    champid_list = rawdata['CHAMPION_ID'].tolist()
    lane_list = rawdata['LANE'].tolist()
    rune_list =  rawdata['RUNE'].tolist()
    win_list = [int(a['win']) for a in rawdata['CHAMPION'].tolist()]

    # 필요 데이터 추출
    need_datas = [{
        'version': version,
        'update_time': update_time,
        'champ_id': champ_id,
        'lane': lane_list[idx],
        'runetype_core': rune_list[idx]['runePrimaryStyle'],
        'core_mainrune': rune_list[idx]['runeCorePerk'],
        'core_rune1': rune_list[idx]['runePrimaryPerk1'],
        'core_rune2': rune_list[idx]['runePrimaryPerk2'],
        'core_rune3': rune_list[idx]['runePrimaryPerk3'],
        'runetype_sub': rune_list[idx]['runeSubStyle'],
        'sub_rune1': rune_list[idx]['runeSubPerk1'],
        'sub_rune2': rune_list[idx]['runeSubPerk2'],
        'shard_offense': rune_list[idx]['runeShardOffense'],
        'shard_flex': rune_list[idx]['runeShardFlex'],
        'shard_defence': rune_list[idx]['runeShardDefense'],
        'win': win_list[idx],
        'game_count': 1
    } for idx, champ_id in enumerate(champid_list)]
    champ_rune = pd.DataFrame(need_datas)

    

    # 그룹화
    result = champ_rune.groupby([
        'version','update_time','champ_id','lane',
        'runetype_core','core_mainrune','core_rune1','core_rune2','core_rune3',
        'runetype_sub','sub_rune1','sub_rune2',
        'shard_offense','shard_flex','shard_defence']).sum().reset_index()

    # 라인 정보에 문제가 있는 레코드 제거
    result = noneDropper(result)
    # 표본수가 부족한 레코드 제거
    # result = ripDropper(result, 50)
    # 승률 정보 추가
    result = winRateAdder(result)
    # 픽률 정보 추가
    result = pickRateAdder(result)
    
    # 테이블의 컬럼에 맞춰 정렬 후 리턴
    result_cols = [
        'version','update_time','champ_id','lane',
        'runetype_core','core_mainrune','core_rune1','core_rune2','core_rune3',
        'runetype_sub','sub_rune1','sub_rune2',
        'shard_offense','shard_flex','shard_defence',
        'pick_rate','game_count','win_rate']

    return result[result_cols]

#### 참고용

In [ ]:
def get_champion_spell():
    now = datetime.datetime.now()
    now = now.strftime('%Y-%m-%d %H:%M:%S')
    
    df = mu.oracle_totalExecute("SELECT version,game_id, champion_id ,participant_number, champion, spell FROM RAWDATA")
    
    df = df.sort_values(['GAME_ID','PARTICIPANT_NUMBER'])
    df_cols = df.columns[4:]
    
    matches = {a:[] for a in df_cols}
    for rec in tqdm(df.iloc):
        for col in [a for a in df_cols]:
            matches[col].append(rawdataLoader(rec[col]))
    for col in [a for a in df_cols]:
        df[col] = matches[col]
    
    champid_lst = df['CHAMPION_ID'].tolist()
    champion_list =  df['CHAMPION'].tolist()
    spell_list =  df['SPELL'].tolist()
    version_list = df['VERSION'].tolist()
    
    print('데이터 추출....')
    
    result=[]
    col = ['version','update_time','championId','lane','win','summoner1Id','summoner2Id']
    for idx,champ in enumerate(tqdm(champion_list)):
        result.append([
            version_list[idx],
            now,
            champid_lst[idx],
            champ['teamPosition'],
            int(champ['win']),
            spell_list[idx]['summoner1Id'],
            spell_list[idx]['summoner2Id'],
        ])
    champ_spell = pd.DataFrame(result,columns=col)
    
    spell_lst = champ_spell[['summoner1Id','summoner2Id']].values.tolist()
    
    result_lst=[]
    
    #sorted 부분
    for i in spell_lst:
         result_lst.append(sorted(i))
    
    df_lst = pd.DataFrame(result_lst,columns=['s1_id','s2_id'])
    
    result_df = champ_spell.join(df_lst)
    print('데이터 그룹화')
    
    
    #승률 그룹바이
    result_df_count = result_df.groupby(['championId','lane','s1_id','s2_id']).count()[['win']].rename(columns={'win':'count'})
    result_df_win = result_df.groupby(['championId','lane','s1_id','s2_id']).sum()[['win']].rename(columns={'win':'total_win'})
    
    result_df_count_win = result_df_count.join(result_df_win)
    
    result_df_count_win.reset_index(inplace=True)
    result_df_count_win['win_rate']= result_df_count_win['total_win']/result_df_count_win['count']
    result_df_count_win['pick_rate']= result_df_count_win['count']/sum(result_df_count_win['count'])
    
    return result_df_count_win

### 챔피언/스킬 (완료!)

In [6]:
def getChampionSkill(rawdata: pd.DataFrame, version: str, update_time: str) -> pd.DataFrame:

    # 속도 개선을 위한 리스트화
    champid_list = rawdata['CHAMPION_ID'].tolist()
    lane_list = rawdata['LANE'].tolist()
    win_list = [int(a['win']) for a in rawdata['CHAMPION']]
    skilltree_list = rawdata['SKILLTREE'].tolist()

    # 필요 데이터 추출
    need_datas = [{
        'version': version,
        'update_time': update_time,
        'champ_id': champ_id,
        'lane': lane_list[idx],
        'skill_build': str(skilltree_list[idx][:15]),
        'level': len(skilltree_list[idx]), # 15레벨 이상의 데이터만 뽑기 위해 임의로 추가한 데이터
        'win': win_list[idx],
        'game_count': 1
    } for idx, champ_id in enumerate(champid_list)]
    champ_skilltree = pd.DataFrame(need_datas)

    # 15레벨 이하의 레코드 제거 후 불필요해진 level 컬럼 제거
    champ_skilltree.drop(champ_skilltree[champ_skilltree['level']<15].index, inplace=True)
    champ_skilltree.drop('level', axis=1, inplace=True)
    champ_skilltree.reset_index(drop=True, inplace=True)

    # 그룹화
    result = champ_skilltree.groupby(['version','update_time','champ_id','lane','skill_build']).sum().reset_index()

    # 라인 정보에 문제가 있는 레코드 제거
    result = noneDropper(result)
    # 표본수가 부족한 레코드 제거
    # result = ripDropper(result, 50)
    # 승률 정보 추가
    result = winRateAdder(result)
    # 픽률 정보 추가
    result = pickRateAdder(result)

    # 테이블의 컬럼에 맞춰 정렬 후 리턴
    result_cols = [
        'version','update_time','champ_id','lane',
        'skill_build','pick_rate','game_count','win_rate']

    return result[result_cols]

### 챔피언/아이템/빌드 (완료!)

In [7]:
def getChampionItemBuild(rawdata: pd.DataFrame, version: str, update_time: str) -> pd.DataFrame:

    # 속도 개선을 위한 리스트화
    champid_list = rawdata['CHAMPION_ID'].to_list()
    lane_list = rawdata['LANE'].to_list()
    win_list = [int(a['win']) for a in rawdata['CHAMPION'].tolist()]
    itemtree_list = rawdata['ITEMTREE'].to_list()
    
    # 필요 데이터 추출
    need_datas = [{
        'version': version,
        'update_time': update_time,
        'champ_id': champ_id,
        'lane': lane_list[idx],
        'item_count': len(itemtree_list[idx]), # 아이템이 3개 이상인 데이터만 뽑기 위해 임의로 추가한 데이터
        'win': win_list[idx],
        'game_count': 1
    } for idx, champ_id in enumerate(champid_list)]
    champ_item_build = pd.DataFrame(need_datas)
    
    # 아이템 갯수가 3개 미만인 레코드 제거 후 불필요해진 item_count 컬럼 제거
    champ_item_build.drop(champ_item_build[champ_item_build['item_count']!=3].index, inplace=True)
    champ_item_build.drop('item_count', axis=1, inplace=True)

    # item1, 2, 3 컬럼 추가
    champ_item_build['item1'] = [itemtree_list[a][0] for a in champ_item_build.index]
    champ_item_build['item2'] = [itemtree_list[a][1] for a in champ_item_build.index]
    champ_item_build['item3'] = [itemtree_list[a][2] for a in champ_item_build.index]
    champ_item_build.reset_index(drop=True, inplace=True)

    # 그룹화
    result = champ_item_build.groupby(['version','update_time','champ_id','lane','item1','item2','item3']).sum().reset_index()
    
    # 라인 정보에 문제가 있는 레코드 제거
    result = noneDropper(result)
    # 표본수가 부족한 레코드 제거
    # result = ripDropper(result, 50)
    # 승률 정보 추가
    result = winRateAdder(result)
    # 픽률 정보 추가
    result = pickRateAdder(result)

    # 테이블의 컬럼에 맞춰 정렬 후 리턴
    result_cols = [
        'version','update_time','champ_id','lane',
        'item1','item2','item3',
        'pick_rate','game_count','win_rate']
    
    return result[result_cols]

### 챔피언/아이템/신발 (완료!)

In [8]:
def getChampionItemBoots(rawdata: pd.DataFrame, version: str, update_time: str) -> pd.DataFrame:
    
    # 속도 개선을 위한 리스트화
    champid_list = rawdata['CHAMPION_ID'].tolist()
    lane_list = rawdata['LANE'].tolist()
    win_list = [int(a['win']) for a in rawdata['CHAMPION'].tolist()]
    item_list = rawdata['ITEM'].tolist()

    # 신발 id 리스트
    boots_id_list = [1001,2422,3006,3009,3020,3047,3111,3117,3158]

    # 신발이 있는 레코드의 인덱스와 해당 레코드에 있는 신발의 id를 리스트로 저장
    boots_list = []
    has_boots_index_list = []
    for idx, items in enumerate(item_list):
        is_boots = 0
        for item in items.values():
            if item in boots_id_list:
                is_boots = item
                has_boots_index_list.append(idx)
                break
        boots_list.append(is_boots)

    # 필요 데이터 추출
    need_datas = [{
        'version': version,
        'update_time': update_time,
        'champ_id': champid_list[idx],
        'lane': lane_list[idx],
        'boots': boots_list[idx],
        'win': win_list[idx],
        'game_count': 1
    } for idx in has_boots_index_list]
    champ_item_boots = pd.DataFrame(need_datas)

    # 그룹화
    result = champ_item_boots.groupby(['version','update_time','champ_id','lane','boots']).sum().reset_index()

    # 라인 정보에 문제가 있는 레코드 제거
    result = noneDropper(result)
    # 표본수가 부족한 레코드 제거
    # result = ripDropper(result, 50)
    # 승률 정보 추가
    result = winRateAdder(result)
    # 픽률 정보 추가
    result = pickRateAdder(result)

    # 테이블의 컬럼에 맞춰 정렬 후 리턴
    result_cols = [
        'version','update_time','champ_id','lane',
        'boots','pick_rate','game_count','win_rate']
    
    return result[result_cols]

### 챔피언/아이템/시작 (완료!)

In [9]:
def getChampionItemStart(rawdata: pd.DataFrame, version: str, update_time: str) -> pd.DataFrame:

    # 속도 개선을 위한 리스트화
    champid_list = rawdata['CHAMPION_ID'].to_list()
    lane_list = rawdata['LANE'].tolist()
    win_list = [int(a['win']) for a in rawdata['CHAMPION']]

    # 정렬용 함수
    def sortItems(items: list) -> list:
        items.sort()
        return items
    # 시작 아이템은 그룹화를 위해 정렬 상태로 리스트화
    start_item_list = [sortItems(item) for item in rawdata['STARTITEM'].tolist()]
    
    # 필요 데이터 추출
    need_datas = [{
        'version': version,
        'update_time': update_time,
        'champ_id': champ_id,
        'lane': lane_list[idx],
        'start_item': str(start_item_list[idx]),
        'win': win_list[idx],
        'game_count': 1
    } for idx, champ_id in enumerate(champid_list)]
    champ_item_start = pd.DataFrame(need_datas)

    # 그룹화
    result = champ_item_start.groupby(['version','update_time','champ_id','lane','start_item']).sum().reset_index()
    
    # 라인 정보에 문제가 있는 레코드 제거
    result = noneDropper(result)
    # 표본수가 부족한 레코드 제거
    # result = ripDropper(result, 50)
    # 승률 정보 추가
    result = winRateAdder(result)
    # 픽률 정보 추가
    result = pickRateAdder(result)

    # 테이블의 컬럼에 맞춰 정렬 후 리턴
    result_cols = [
        'version','update_time','champ_id','lane',
        'start_item','pick_rate','game_count','win_rate']
    
    return result[result_cols]

### 챔피언/스펠 (완료!)

In [10]:
def getChampionSpell(rawdata: pd.DataFrame, version: str, update_time: str):
    
    champid_list = rawdata['CHAMPION_ID'].tolist()
    lane_list = rawdata['LANE'].tolist()
    win_list = [int(a['win']) for a in rawdata['CHAMPION']]
    spell_list =  rawdata['SPELL'].tolist()
    
    need_datas = [{
        'champ_id': champ_id,
        'lane': lane_list[idx],
        'spell1': spell_list[idx]['summoner1Id'],
        'spell2': spell_list[idx]['summoner2Id'],
        'win': win_list[idx]
    } for idx, champ_id in enumerate(champid_list)]
    champ_spell = pd.DataFrame(need_datas)
    
    spell_list = champ_spell[['spell1','spell2']].values.tolist()
    
    result_list=[]
    for i in spell_list:
         result_list.append(sorted(i))
    
    rawdata_list = pd.DataFrame(result_list,columns=['s1_id','s2_id'])
    
    result_rawdata = champ_spell.join(rawdata_list)
    
    # return result_rawdata
    
    #승률 그룹바이
    result_rawdata_count = result_rawdata.groupby(['champ_id','lane','s1_id','s2_id']).count()[['win']].rename(columns={'win':'game_count'})
    result_rawdata_win = result_rawdata.groupby(['champ_id','lane','s1_id','s2_id']).sum()[['win']]
    
    result_rawdata_count_win = result_rawdata_count.join(result_rawdata_win)
    
    result_rawdata_count_win.reset_index(inplace=True)
    # 문제 데이터 제거
    result_rawdata_count_win = noneDropper(result_rawdata_count_win)
    # 승률 추가
    result_rawdata_count_win = winRateAdder(result_rawdata_count_win)
    # 픽률 추가
    result_rawdata_count_win = pickRateAdder(result_rawdata_count_win)
    
    result_rawdata_count_win['version'] = version
    result_rawdata_count_win['update_time'] = update_time
    result_rawdata_count_win = result_rawdata_count_win[['version','update_time','champ_id','lane','s1_id','s2_id','pick_rate','game_count','win_rate']]\
        .rename(columns={'s1_id':'spell1', 's2_id':'spell2'})
    return result_rawdata_count_win

### 챔피언/매치업 (완료!)

In [11]:
def getChampionMatchup(rawdata: pd.DataFrame, version: str, update_time: str) -> pd.DataFrame:

    #list화
    gameid_list = rawdata['GAME_ID'].tolist()
    championid_lst = rawdata['CHAMPION_ID'].tolist()
    champion_list =  rawdata['CHAMPION'].tolist()
    kda_list = rawdata['KDA'].tolist()
    damage_list = rawdata['DAMAGE'].tolist()

    # print('데이터 추출....')
    result=[]
    col=['gameid','championId','lane','win','teamId','kda','killParticipation','totalDamageDealtToChampions']
    for idx,champ in enumerate(champion_list):
        result.append([
            gameid_list[idx],
            championid_lst[idx],
            champ['teamPosition'],
            int(champ['win']),
            champ['teamId'],
            round(kda_list[idx]['kda'],2),
            round(kda_list[idx]['killParticipation'],2),
            damage_list[idx]['totalDamageDealtToChampions']
            ])
    champ_match = pd.DataFrame(result,columns=col)
    
    # print('그룹화중..')
    champ_match_100 = champ_match[champ_match['teamId']==100]
    champ_match_200 = champ_match[champ_match['teamId']==200]
    
    champ_match_200 = champ_match_200.rename(columns={'championId':'opponent_Id','win':'opponent_win','teamId':'opponent_teamId',
                                                 'kda':'opponent_kda','killParticipation':'opponent_killParticipation',
                                                 'totalDamageDealtToChampions':'opponent_totalDamageDealtToChampions'})
    
    

    match_info_rawdata = pd.merge(champ_match_100,champ_match_200, on=['gameid','lane'])
    
    #챔프아이디,상대아이디 리스트화 비교위해
    match_info_rawdata_lst = match_info_rawdata[['championId','opponent_Id']].values.tolist()
    
    result_lst=[]
    for i in match_info_rawdata_lst:
        result_lst.append(sorted(i))
    
    rawdata_lst = pd.DataFrame(result_lst,columns=['f_id','s_id'])
    
    result_rawdata = match_info_rawdata.join(rawdata_lst)
    
    #승률 그룹바이
    result_rawdata_count = result_rawdata.groupby(['lane','f_id','s_id']).count()[['win']].rename(columns={'win':'count'})
    result_rawdata_win = result_rawdata.groupby(['lane','f_id','s_id']).sum()[['win']].rename(columns={'win':'total_win'})
    
    result_rawdata_count_win = result_rawdata_count.join(result_rawdata_win)
    result_rawdata_count_win.reset_index(inplace=True)
    result_rawdata_count_win['win_rate']= round((result_rawdata_count_win['total_win']/result_rawdata_count_win['count']) * 100,2)
    
    #통계 그룹바이
    result_rawdata_maen = result_rawdata.groupby(['lane','f_id','s_id']).mean().reset_index().round(2)
    result_rawdata_maen=result_rawdata_maen[['lane','f_id','kda','killParticipation','totalDamageDealtToChampions','s_id','opponent_kda','opponent_killParticipation','opponent_totalDamageDealtToChampions']]
    
    #통계 머지
    last_rawdata = pd.merge(result_rawdata_maen,result_rawdata_count_win,on=['lane',"f_id","s_id"])
    last_rawdata.rename(columns={'f_id':'champion_id','s_id':'opponent_id'},inplace=True)

    last_rawdata['version'] = version
    last_rawdata['update_time'] = update_time
    last_rawdata.rename(columns={
        'champion_id':'champ_id_x','opponent_id':'champ_id_y','kda':'kda_x','opponent_kda':'kda_y',
        'killParticipation':'killparticipation_x','opponent_killParticipation':'killparticipation_y',
        'totalDamageDealtToChampions':'total_champ_damage_x','opponent_totalDamageDealtToChampions':'total_champ_damage_y',
        'count':'matchup_count','win_rate':'matchup_win_rate'}, inplace=True)

    result_cols = [
        'version','update_time','lane',
        'champ_id_x','champ_id_y','matchup_count','matchup_win_rate',
        'kda_x','kda_y','killparticipation_x','killparticipation_y','total_champ_damage_x','total_champ_damage_y']

    return last_rawdata[result_cols]

## 작동 테스트

In [17]:
def updateIntervalTables(version: str):
    
    # print('>> RawData 테이블 무결성 상태 확인 중...')
    # mu.rawdataIntegrityKeeper()

    print('>> RawData에서 데이터 가져오는 중...')
    need_cols = ['VERSION','GAME_ID','PARTICIPANT_NUMBER','CHAMPION_ID','LANE','GAME','CHAMPION','SPELL','SKILLTREE','RUNE','STARTITEM','ITEMTREE','ITEM','KDA','DAMAGE']
    update_time = time.strftime('%Y-%m-%d %H:%M:%S')
    rawdata = mu.oracle_totalExecute(f"SELECT {','.join(need_cols)} FROM RAWDATA WHERE VERSION = '{version}'", debug_print=False)
    print('>> 총 레코드 수:', len(rawdata))

    print('>> 가져온 데이터 정렬 및 파싱 중...')
    rawdata.sort_values(['GAME_ID', 'PARTICIPANT_NUMBER'], ascending=[False, True], ignore_index=True, inplace=True)
    rawdata = parseRawdata(rawdata, 5)
    
    print('>> getChampionInfo() 실행 중...')
    info = getChampionInfo(rawdata, version, update_time)
    print(f">> Info 레코드 수: {len(info)}")
    print('>> CHAMPION_INFO 삽입 중...')
    mu.insertDataFrameIntoTable(info, 'CHAMPION_INFO', True)
    print()
    
    print('>> getChampionRune() 실행 중...')
    rune = getChampionRune(rawdata, version, update_time)
    # 보조 룬 타입이 0인 소수의 데이터에 대한 처리
    rune.drop(rune[rune['runetype_sub']==0].index, inplace=True)
    print(f">> Rune 레코드 수: {len(rune)}")
    print('>> CHAMPION_RUNE 삽입 중...')
    mu.insertDataFrameIntoTable(rune, 'CHAMPION_RUNE', True)
    print()

    print('>> getChampionSkill() 실행 중...')
    skill = getChampionSkill(rawdata, version, update_time)
    print(f">>  레코드 수: {len(skill)}")
    print('>> CHAMPION_SKILL 삽입 중...')
    mu.insertDataFrameIntoTable(skill, 'CHAMPION_SKILL', True)
    print()

    print('>> getChampionItemBuild() 실행 중...')
    build = getChampionItemBuild(rawdata, version, update_time)
    print(f">>  레코드 수: {len(build)}")
    print('>> CHAMPION_ITEM_BUILD 삽입 중...')
    mu.insertDataFrameIntoTable(build, 'CHAMPION_ITEM_BUILD', True)
    print()

    print('>> getChampionItemBoots() 실행 중...')
    boots = getChampionItemBoots(rawdata, version, update_time)
    print(f">>  레코드 수: {len(boots)}")
    print('>> CHAMPION_ITEM_BOOTS 삽입 중...')
    mu.insertDataFrameIntoTable(boots, 'CHAMPION_ITEM_BOOTS', True)
    print()

    print('>> getChampionItemStart() 실행 중...')
    start = getChampionItemStart(rawdata, version, update_time)
    print(f">>  레코드 수: {len(start)}")
    print('>> CHAMPION_ITEM_START 삽입 중...')
    mu.insertDataFrameIntoTable(start, 'CHAMPION_ITEM_START', True)
    print()

    print('>> getChampionSpell() 실행 중...')
    spell = getChampionSpell(rawdata, version, update_time)
    print(f">>  레코드 수: {len(spell)}")
    print('>> CHAMPION_SPELL 삽입 중...')
    mu.insertDataFrameIntoTable(spell, 'CHAMPION_SPELL', True)
    print()

    print('>> getChampionMatchup() 실행 중...')
    matchup = getChampionMatchup(rawdata, version, update_time)
    print(f">>  레코드 수: {len(matchup)}")
    print('>> CHAMPION_MATCHUP 삽입 중...')
    mu.insertDataFrameIntoTable(matchup, 'CHAMPION_MATCHUP', True)
    print()

    print('>> 모든 인터벌 테이블 삽입 완료!')

In [ ]:
updateIntervalTables(version='13.9.506.4846')